In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn import preprocessing
import pickle
import warnings

warnings.filterwarnings('ignore')
import ipywidgets as widgets
from IPython.display import display, clear_output
import datetime
import requests
import math

high_model = pickle.load(open('high_model', 'rb'))
low_model = pickle.load(open('low_model', 'rb'))
pressure_model = pickle.load(open('pressure_model', 'rb'))


def get_ip():
    api_url = "https://ipinfo.io/json"
    response = requests.get(api_url)
    ip = response.json()["ip"].split(",")
    return str(ip[0])


def get_wet(temp, hum):
    return temp * math.atan(0.152 * (hum + 8.3136) ** 2) + math.atan(temp + hum) - math.atan(
        hum - 1.6763) + 0.00391838 * hum ** (3 / 2) * math.atan(.0231 * hum) - 4.686

def normalization(x, max, min):
    print(min, max)
    return (x - min) / (max - min)


def denormalization(x, max, min):
    print(min, max)
    return x * (max - min) + min


days = []
temp_mins = []
temp_maxs = []
winds = []
humidities = []
wet_lows = []
wet_highs = []
pressures = []

normalize = pickle.load(open('normalization3.pickle', 'rb'))

def get_new_api_info(norm):
    now = datetime.datetime.now()
    days.append(str(now.replace(hour=0, minute=0, second=0, microsecond=0) - datetime.timedelta(days=0))[:11])
    days.append(str(now.replace(hour=0, minute=0, second=0, microsecond=0) - datetime.timedelta(days=1))[:11])
    days.append(str(now.replace(hour=0, minute=0, second=0, microsecond=0) - datetime.timedelta(days=2))[:11])
    days.append(str(now.replace(hour=0, minute=0, second=0, microsecond=0) - datetime.timedelta(days=3))[:11])

    ip = get_ip()
    built_api_url = "http://api.weatherapi.com/v1/history.json?key=fa024d2a350b4df484c42502232603 &q=" + ip + "&dt="

    count = 0
    for day in days:
        response = requests.get(built_api_url + day)
        j = response.json()
        min = normalization(j["forecast"]["forecastday"][0]["day"]["mintemp_f"], norm['low'][1], norm['low'][0])
        temp_mins.append(min)
        max = normalization(j["forecast"]["forecastday"][0]["day"]["maxtemp_f"], norm['high'][1], norm['high'][0])
        temp_maxs.append(max)
        wind = normalization(j["forecast"]["forecastday"][0]["day"]["maxwind_mph"], norm['wind_speed'][1], norm['wind_speed'][0])
        winds.append(wind)
        humidity = normalization(j["forecast"]["forecastday"][0]["day"]["avghumidity"], norm['humidity'][1], norm['humidity'][0])
        humidities.append(humidity)
        pressure = normalization(j["forecast"]["forecastday"][0]["hour"][0]["pressure_in"], norm['pressure'][1], norm['pressure'][0])
        pressures.append(pressure)
        wet_highs.append(normalization(get_wet(max, humidity), norm['wet_bulb_high'][1], norm['wet_bulb_high'][0]))
        wet_lows.append(normalization(get_wet(min, humidity), norm['wet_bulb_low'][1], norm['wet_bulb_low'][0]))
        count += 1

get_new_api_info(normalize)


def get_past_days():
    return pd.DataFrame([[temp_maxs[0], temp_maxs[1], temp_maxs[2], temp_maxs[3], temp_mins[0], temp_mins[1],
                          temp_mins[2], temp_mins[3], wet_highs[0], wet_highs[1], wet_highs[2], wet_highs[3],
                          wet_lows[0], wet_lows[1], wet_lows[2], wet_lows[3], humidities[0], humidities[1],
                          humidities[2], humidities[3], winds[0], winds[1], winds[2], winds[3], pressures[0],
                          pressures[1], pressures[2], pressures[3]]], columns=['High-1', 'High-2', 'High-3', 'High-4',
                                                                               'Low-1', 'Low-2', 'Low-3', 'Low-4',
                                                                               'WetBulb_High-1', 'WetBulb_High-2',
                                                                               'WetBulb_High-3', 'WetBulb_High-4',
                                                                               'WetBulb_Low-1', 'WetBulb_Low-2',
                                                                               'WetBulb_Low-3', 'WetBulb_Low-4',
                                                                               'Humidity-1', 'Humidity-2', 'Humidity-3',
                                                                               'Humidity-4',
                                                                               'WindSpeed-1', 'WindSpeed-2',
                                                                               'WindSpeed-3', 'WindSpeed-4',
                                                                               'Pressure-1', 'Pressure-2', 'Pressure-3',
                                                                               'Pressure-4'])

def high_widget():
    high_prediction = high_model.predict(get_past_days())[0]
    high_prediction = denormalization(high_prediction, normalize['high'][1], normalize['high'][0])
    return widgets.FloatText(value=high_prediction)


def low_widget():
    low_prediction = low_model.predict(get_past_days())[0]
    low_prediction = denormalization(low_prediction, normalize['low'][1], normalize['low'][0])
    return widgets.FloatText(value=low_prediction)


def pressure_widget():
    pressure_prediction = pressure_model.predict(get_past_days())[0]
    pressure_prediction = denormalization(pressure_prediction, normalize['pressure'][1], normalize['pressure'][0])
    return widgets.FloatText(value=pressure_prediction)


text_1 = widgets.HTML(value="<h1>The high for tomorrow is: </h1>")
text_2 = widgets.HTML(value="<h2>The low for tomorrow is: </h2>")
text_3 = widgets.HTML(value="<h3>The pressure for tomorrow is: </h3>")


button_predict = widgets.Button(
    description='Predict Tomorrow',
    tooltip='Predict',
    style={'description_width': 'initial'}
)


def set_page():
    vbox_text = widgets.VBox([text_1, high_widget(), text_2, low_widget(), text_3, pressure_widget(), vbutton])
    page = widgets.HBox([vbox_text])
    display(page)


def on_button_clicked(event):
    set_page()


vbutton = widgets.VBox([button_predict])
set_page()
button_predict.on_click(on_button_clicked)